# 데이터 로드 및 사전처리
## 6. unicode 데이터 로드
#### 참고사이트 : https://www.tensorflow.org/tutorials/load_data/unicode?hl=ko

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

In [2]:
# tf.string 데이터 타입
# tensorflow의 dtype은 byte 문자열로 이루어진 tensor 생성되며,
# 유니코드 문자열은 기본적으로 utf-8로 인코딩.
tf.constant(u"Thanks 😊")

<tf.Tensor: id=0, shape=(), dtype=string, numpy=b'Thanks \xf0\x9f\x98\x8a'>

In [3]:
tf.constant([u"You're", u"welcome!"]).shape

TensorShape([2])

In [7]:
# 유니코드 표현
# 1) string 스칼라 = 문자 인코딩을 사용해 인코딩
# 2) int32 벡터 = 위치마다 개별 코드 포인트를 포함
# 아래 3가지 값이 모두 중국어로 '언어처리'를 의미

text_utf8 = tf.constant(u"语言处理") # UTF-8로 인코딩된 string 스칼라로 표현한 유니코드 문자열
print(text_utf8)

text_utf16be = tf.constant(u"语言处理".encode("UTF-16-BE")) # UTF-16-BE로 인코딩된 string 스칼라로 표현한 유니코드 문자열
print(text_utf16be)

text_chars = tf.constant([ord(char) for char in u"语言处理"]) # 유니코드 코드 포인트의 벡터로 표현한 유니코드 문자열
print(text_chars)

tf.Tensor(b'\xe8\xaf\xad\xe8\xa8\x80\xe5\xa4\x84\xe7\x90\x86', shape=(), dtype=string)
tf.Tensor(b'\x8b\xed\x8a\x00Y\x04t\x06', shape=(), dtype=string)
tf.Tensor([35821 35328 22788 29702], shape=(4,), dtype=int32)


In [11]:
# 표현 간의 변환
# 1) tf.strings.unicode_decode: 인코딩된 string 스칼라를 코드 포인트의 벡터로 변환
# 2) tf.strings.unicode_encode: 코드 포인트의 벡터를 인코드된 string 스칼라로 변환
# 3) tf.strings.unicode_transcode: 인코드된 string 스칼라를 다른 인코딩으로 변환
print(tf.strings.unicode_decode(text_utf8, input_encoding='UTF-8'))
print(tf.strings.unicode_encode(text_chars, output_encoding='UTF-8'))
print(tf.strings.unicode_transcode(text_utf8, input_encoding='UTF8', output_encoding='UTF-16-BE'))

tf.Tensor([35821 35328 22788 29702], shape=(4,), dtype=int32)
tf.Tensor(b'\xe8\xaf\xad\xe8\xa8\x80\xe5\xa4\x84\xe7\x90\x86', shape=(), dtype=string)
tf.Tensor(b'\x8b\xed\x8a\x00Y\x04t\x06', shape=(), dtype=string)


In [12]:
# Batch demension
# UTF-8 인코딩된 문자열로 표현한 유니코드 문자열의 배치
batch_utf8 = [s.encode('UTF-8') for s in
              [u'hÃllo', u'What is the weather tomorrow', u'Göödnight', u'😊']]
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8, input_encoding='UTF-8')
for sentence_chars in batch_chars_ragged.to_list():
    print(sentence_chars)

[104, 195, 108, 108, 111]
[87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116, 104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]
[71, 246, 246, 100, 110, 105, 103, 104, 116]
[128522]


In [15]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

[[   104    195    108    108    111     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [    87    104     97    116     32    105    115     32    116    104
     101     32    119    101     97    116    104    101    114     32
     116    111    109    111    114    114    111    119]
 [    71    246    246    100    110    105    103    104    116     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [128522     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]]


In [16]:
# sparse는 희소 tensor를 의미
batch_chars_sparse = batch_chars_ragged.to_sparse()

In [17]:
# vector를 scalar로 변환(숫자 -> 문자)
tf.strings.unicode_encode([[99, 97, 116], [100, 111, 103], [ 99, 111, 119]],
                          output_encoding='UTF-8')

<tf.Tensor: id=275, shape=(3,), dtype=string, numpy=array([b'cat', b'dog', b'cow'], dtype=object)>

In [18]:
tf.strings.unicode_encode(batch_chars_ragged, output_encoding='UTF-8') # 숫자를 문자로

<tf.Tensor: id=276, shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

In [19]:
# 길이가 다른 여러 문자열을 인코딩 할 때는 tf.RaggedTensor(비정형 텐서)로 바꿔주기
# 예를 들면 패딩 또는 sparse(희소, 드문) tensor를 의미
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8')

<tf.Tensor: id=355, shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

In [20]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1),
    output_encoding='UTF-8')

<tf.Tensor: id=428, shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

In [23]:
# 유니코드 연산
# 길이 = tf.strings.length 연산은 unit 인자를 가짐(기본은 byte이지만, utf8_char이나 utf16_char으로 변환 가능)

thanks = u'Thanks 😊'.encode('UTF-8') # UTF8에서 이모티콘은 4바이트를 차지
num_bytes = tf.strings.length(thanks).numpy() # 띄어쓰기 포함 6 + 1 + 4 = 11bytes
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} 바이트; {}개의 UTF-8 문자'.format(num_bytes, num_chars))

11 바이트; 8개의 UTF-8 문자


In [28]:
# substring(매개변수, pos, len) -> 매개변수의 pos 위치로부터 len개 만큼의 bytes 반환
# tf.strings.substr 연산은 unit 매개변수를 사용해 pos와 len으로 컨트롤
tf.strings.substr(thanks, pos=7, len=1).numpy() # 기본: unit='BYTE' / len=1이면 바이트 하나 반환

b'\xf0'

In [29]:
# unit='UTF8_CHAR'로 지정하면 4 바이트인 문자 하나 반환
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

b'\xf0\x9f\x98\x8a'


In [32]:
# unicode 문자열 분리(split 함수 활용)
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

array([b'T', b'h', b'a', b'n', b'k', b's', b' ', b'\xf0\x9f\x98\x8a'],
      dtype=object)

In [33]:
# offset(상대주소)
# tf.strings.unicode_decode로 만든(문자->숫자) tensor를 맞추려면 각 문자의 시작위치 offset을 알아야 함.
# tf.strings.unicode_decode_with_offsets은 unicode_decode와 비슷하지만 각 문자의 시작 오프셋을 포함한 두 번째 텐서를 반환
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u"🎈🎉🎊", 'UTF-8')

for (codepoint, offset) in zip(codepoints.numpy(), offsets.numpy()):
    print("바이트 오프셋 {}: 코드 포인트 {}".format(offset, codepoint))

바이트 오프셋 0: 코드 포인트 127880
바이트 오프셋 4: 코드 포인트 127881
바이트 오프셋 8: 코드 포인트 127882


In [34]:
# unicode script
# 각 unicode의 codepoint는 script라 부르는 하나의 코드포인트 집합에 속함.
# 문자의 script는 어떤 언어인지 맞추는데 도움
# ex)  'Б'가 키릴(Cyrillic) 스크립트라는 것을 알고 있으면 이 문자가 포함된 텍스트는 (러시아어나 우크라이나어 같은) 슬라브 언어라는 것을 알 수 있음
uscript = tf.strings.unicode_script([33464, 1041])  # ['芸', 'Б']

print(uscript.numpy())  # [17, 8] == [USCRIPT_HAN, USCRIPT_CYRILLIC]

[17  8]


In [35]:
# script 연산은 코드포인트의 다차원(tf.Tensor나 tf.RaggedTensor)에 적용 가능
print(tf.strings.unicode_script(batch_chars_ragged))

<tf.RaggedTensor [[25, 25, 25, 25, 25], [25, 25, 25, 25, 0, 25, 25, 0, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25, 25], [25, 25, 25, 25, 25, 25, 25, 25, 25], [0]]>


### 예제 = 간단한 segmentaion(분할)

**분할(segmentation)** 은 텍스트를 단어와 같은 단위로 나누는 작업.   
공백 문자가 단어를 나누는 구분자로 사용되는 경우는 쉽지만, 중국어나 일본어처럼 공백을 사용하지 않는 언어나 독일어 처럼 단어를 길게 조합하는 언어는 의미를 분석하기 위한 분할 과정이 꼭 필요.   
웹 텍스트에는 "NY株価"(New York Stock Exchange)처럼 여러 가지 언어와 스크립트가 섞여 있는 경우가 많음.

script의 변화를 단어 경계로 근사하여 ML 모델 사용 없이 대략적인 segmentation을 수행할 수 있음.   
위에서 언급된 "NY株価"의 예와 같은 문자열에 적용됨.   
다양한 스크립트의 공백 문자를 모두 USCRIPT_COMMON(실제 텍스트의 스크립트 코드와 다른 특별한 스크립트 코드)으로 분류하기 때문에 공백을 사용하는 대부분의 언어들에도 적용 가능!

In [43]:
# dtype: string; shape: [num_sentences]
sentence_texts = [u'Hello, world.', u'世界こんにちは', u'안녕하세요 반갑습니다!']

In [44]:
# 위의 문장으로 codepoint로 decode하고, 각 문자에 대한 script 식별자 찾기

# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_codepoint[i, j]는
# i번째 문장 안에 있는 j번째 문자에 대한 코드 포인트 입니다.
sentence_char_codepoint = tf.strings.unicode_decode(sentence_texts, 'UTF-8')
print(sentence_char_codepoint)

# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_codepoint[i, j]는 
# i번째 문장 안에 있는 j번째 문자의 유니코드 스크립트 입니다.
sentence_char_script = tf.strings.unicode_script(sentence_char_codepoint) 
print(sentence_char_script) # 한국어의 unicode script는 18인듯

<tf.RaggedTensor [[72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 46], [19990, 30028, 12371, 12435, 12395, 12385, 12399], [50504, 45397, 54616, 49464, 50836, 32, 48152, 44049, 49845, 45768, 45796, 33]]>
<tf.RaggedTensor [[25, 25, 25, 25, 25, 0, 0, 25, 25, 25, 25, 25, 0], [17, 17, 20, 20, 20, 20, 20], [18, 18, 18, 18, 18, 0, 18, 18, 18, 18, 18, 0]]>


In [45]:
# script 식별자를 통해 단어 경계를 추가 할 위치 결정
# 각 문장의 시작과 이전 문자의 script가 다른 문자에 경계 추가

# dtype: bool; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_starts_word[i, j]는 
# i번째 문장 안에 있는 j번째 문자가 단어의 시작이면 True 입니다.
sentence_char_starts_word = tf.concat(
    [tf.fill([sentence_char_script.nrows(), 1], True),
     tf.not_equal(sentence_char_script[:, 1:], sentence_char_script[:, :-1])],
    axis=1)

# dtype: int64; shape: [num_words]
#
# word_starts[i]은 (모든 문장의 문자를 일렬로 펼친 리스트에서)
# i번째 단어가 시작되는 문자의 인덱스 입니다.
word_starts = tf.squeeze(tf.where(sentence_char_starts_word.values), axis=1)
print(word_starts) # , . (띄어쓰기) ! 등

tf.Tensor([ 0  5  7 12 13 15 20 25 26 31], shape=(10,), dtype=int64)


In [46]:
# 위에서 나온 시작 offset을 사용해 전체 batch에 있는 단어 list를 담은 RaggedTensor(비정형) 생성

# dtype: int32; shape: [num_words, (num_chars_per_word)]
#
# word_char_codepoint[i, j]은 
# i번째 단어 안에 있는 j번째 문자에 대한 코드 포인트 입니다.
word_char_codepoint = tf.RaggedTensor.from_row_starts(
    values=sentence_char_codepoint.values,
    row_starts=word_starts)
print(word_char_codepoint)

<tf.RaggedTensor [[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46], [19990, 30028], [12371, 12435, 12395, 12385, 12399], [50504, 45397, 54616, 49464, 50836], [32], [48152, 44049, 49845, 45768, 45796], [33]]>


In [47]:
# 마지막으로 codepoint RaggedTensor를 문장으로 다시 나누기

# dtype: int64; shape: [num_sentences]
#
# sentence_num_words[i]는 i번째 문장 안에 있는 단어의 수입니다.
sentence_num_words = tf.reduce_sum(
    tf.cast(sentence_char_starts_word, tf.int64),
    axis=1)

# dtype: int32; shape: [num_sentences, (num_words_per_sentence), (num_chars_per_word)]
#
# sentence_word_char_codepoint[i, j, k]는 i번째 문장 안에 있는
# j번째 단어 안의 k번째 문자에 대한 코드 포인트입니다.
sentence_word_char_codepoint = tf.RaggedTensor.from_row_lengths(
    values=word_char_codepoint,
    row_lengths=sentence_num_words)
print(sentence_word_char_codepoint)

<tf.RaggedTensor [[[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46]], [[19990, 30028], [12371, 12435, 12395, 12385, 12399]], [[50504, 45397, 54616, 49464, 50836], [32], [48152, 44049, 49845, 45768, 45796], [33]]]>


In [48]:
# 최종 결과를 읽기 쉽게 utf-8 문자열로 인코딩(숫자->문자)
tf.strings.unicode_encode(sentence_word_char_codepoint, 'UTF-8').to_list()

[[b'Hello', b', ', b'world', b'.'],
 [b'\xe4\xb8\x96\xe7\x95\x8c',
  b'\xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf'],
 [b'\xec\x95\x88\xeb\x85\x95\xed\x95\x98\xec\x84\xb8\xec\x9a\x94',
  b' ',
  b'\xeb\xb0\x98\xea\xb0\x91\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4',
  b'!']]